Structure of the visajourney website is that the data is stored in the class called "pme-main" and structure is as follows:

\<table class="pme-main"> \
tbody \
\<tr> \
\<th> \<b>"Header Name here" <\b> \</th>  

\<tr> \
	\<td> \<a> "Nick name of an applicant" \</a>\</td> \
	\<td> "visa type" \</td>  \
	\<td> \<a>"city name" \</a> \</td> \
	the rest of the inputs are all in \<td> \</td> \ 
\</tr>

Then the next person starts in another \<tr>\</tr>

In [46]:
%reset -f

In [200]:
%reset -f
# standard library 
import requests
import csv, re
import time 
import cProfile
import pstats
# external library 
from bs4 import BeautifulSoup
from tqdm import tqdm 


In [203]:
url = "https://www.visajourney.com/timeline/aoslist.php"

with requests.Session() as s:
    print("Session initialized")
    data = []
    soup = BeautifulSoup(s.get(url).text, "html.parser")
    # This will gives the span with class pagelinklast, and then find the child element a
    last_page_url = soup.find("span", {"class": "pagelinklast"}).find("a")
    # Then find the href, and split by & 
    last_page = last_page_url['href'].split("&")
    for item in last_page:
        # The last page parameter is in op4
        if "op4" in item:
            last_page_num = int(item.split("=")[1])
    target_table = "pme-main"
    table = soup.find(class_=target_table)
    # print(table)
    headers = []
    data = []
    # Make list of headers 
    for col in table.find_all('th'):
        headers.append(col.text.strip())
    last_page_num = 5
    # iterate over pages 
    for page in range(1,last_page_num+1):
        current_page = url + f"?op4={page}"
        print(current_page)
        soup = BeautifulSoup(s.get(current_page).text,"html.parser")
        table = soup.find(class_ = target_table)
        counter = 0 
        for row in table.find_all("tr"):
            # Need to skip the header tr 
            if (counter == 0):
                counter = 1 
                continue 
            t_row = {}
            for col, header in zip(row.find_all('td'), headers):
                t_row[header] = col.text.strip()
            data.append(t_row)
    keys = data[0].keys()
    file_name = 'visajourney.csv'
    with open(file_name, 'w', encoding='utf-8', newline="") as data_file:
        dict_writer = csv.DictWriter(data_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data) 


Session initialized
https://www.visajourney.com/timeline/aoslist.php?op4=1
https://www.visajourney.com/timeline/aoslist.php?op4=2
https://www.visajourney.com/timeline/aoslist.php?op4=3
https://www.visajourney.com/timeline/aoslist.php?op4=4
https://www.visajourney.com/timeline/aoslist.php?op4=5
